In [7]:
txt = input()  

hello i am sad today


In [8]:
from textblob import TextBlob
textb = TextBlob(txt)

In [9]:
textb.sentiment

Sentiment(polarity=-0.5, subjectivity=1.0)

In [30]:
##lets compare the sentiment of vader
import pandas as pd
import numpy as np
import re
df = pd.read_csv('uncleanedds.csv')
df = df.drop(columns = ["Unnamed: 0"])
df['no_punc_text'] = df['full_text']
df = df[~df['no_punc_text'].str.startswith('RT')]
df = df[~df['no_punc_text'].str.startswith('LIVE')]
df = df[~df['no_punc_text'].str.startswith('ICYMI')]
df = df[~df['no_punc_text'].str.startswith('WATCH')]
def removeatusernames(tweet):
    return re.sub(r'(@[A-Za-z0-9]+)','',tweet)
def removelinks(tweet):
    return re.sub(r'http\S+','',tweet)
def removeemojis(tweet):
    return tweet.encode('ascii', 'ignore').decode('ascii')
def removehashtags(tweet):
    return re.sub(r'(#[A-Za-z0-9]+)','',tweet)
def removePuncs(tweet):
    return re.sub(r'[^\w\s]','', tweet)

In [11]:
df['no_punc_text'] = df['no_punc_text'].apply(removelinks).apply(removeatusernames).apply(removehashtags)

In [12]:
df = df[~df['no_punc_text'].str.startswith(' ')]
df['no_punc_text'] = df['no_punc_text'].str.lower()

In [13]:
#VADER
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
def sentiment_analyzer_scores_1(sentence):
    score = analyser.polarity_scores(sentence)
    return score['compound']
df['vader_pol'] = df['full_text'].apply(sentiment_analyzer_scores_1)

In [14]:
df['vader_sentiment'] = [1 if x >= 0 else 0 for x in df['vader_pol']]


In [15]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = 'word',lowercase = False)
features = vectorizer.fit_transform(df['no_punc_text'])
features_nd = features.toarray()

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(
        features_nd, 
        df['vader_sentiment'],
        train_size=0.80)

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [17]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()
log_model = log_model.fit(X=X_train, y=y_train)
y_pred = log_model.predict(X_test)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [18]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9092783505154639


In [19]:
import pandas as pd
import numpy as np
import re
tb_df = pd.read_csv('uncleanedds.csv')
tb_df = tb_df.drop(columns = ["Unnamed: 0"])
tb_df['no_punc_text'] = tb_df['full_text']
tb_df = tb_df[~tb_df['no_punc_text'].str.startswith('RT')]
tb_df = tb_df[~tb_df['no_punc_text'].str.startswith('LIVE')]
tb_df = tb_df[~tb_df['no_punc_text'].str.startswith('ICYMI')]
tb_df = tb_df[~tb_df['no_punc_text'].str.startswith('WATCH')]
def removeatusernames(tweet):
    return re.sub(r'(@[A-Za-z0-9]+)','',tweet)
def removelinks(tweet):
    return re.sub(r'http\S+','',tweet)
def removeemojis(tweet):
    return tweet.encode('ascii', 'ignore').decode('ascii')
def removehashtags(tweet):
    return re.sub(r'(#[A-Za-z0-9]+)','',tweet)
def removePuncs(tweet):
    return re.sub(r'[^\w\s]','', tweet)
tb_df['no_punc_text'] = tb_df['no_punc_text'].apply(removelinks).apply(removeatusernames).apply(removehashtags)
tb_df = tb_df[~tb_df['no_punc_text'].str.startswith(' ')]
tb_df['no_punc_text'] = tb_df['no_punc_text'].str.lower()


In [20]:
from textblob import TextBlob
sentiment_objects = [TextBlob(tweet) for tweet in tb_df['no_punc_text']]
sentiment_values = [[tweet.sentiment.polarity, str(tweet)] for tweet in sentiment_objects]
sentiment_df = pd.DataFrame(sentiment_values, columns=["polarity", "text"])
sentiment_df['sentiment'] = [1 if x >= 0 else 0 for x in sentiment_df['polarity']]

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = 'word',lowercase = False)
features = vectorizer.fit_transform(sentiment_df['text'])
features_nd = features.toarray()
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test  = train_test_split(
        features_nd, 
        sentiment_df['sentiment'],
        train_size=0.80)
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()
log_model = log_model.fit(X=X_train, y=y_train)
y_pred = log_model.predict(X_test)

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [22]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9381443298969072


In [23]:
listoftweets = ['The amount of time I put into to study for one midterm here is the amount of time I put in to study for two AP tests in high school... and you’re telling me I didn’t get an A??? A scam','Really not trynna go back to school tomorrow smh','i think i have, like, extra feelings']

In [29]:
tb_df = pd.read_csv('uncleanedds.csv')
tb_df = tb_df.drop(columns = ["Unnamed: 0"])
tb_df['no_punc_text'] = tb_df['full_text']
tb_df = tb_df[~tb_df['no_punc_text'].str.startswith('RT')]
tb_df = tb_df[~tb_df['no_punc_text'].str.startswith('LIVE')]
tb_df = tb_df[~tb_df['no_punc_text'].str.startswith('ICYMI')]
tb_df = tb_df[~tb_df['no_punc_text'].str.startswith('WATCH')]
def removeatusernames(tweet):
    return re.sub(r'(@[A-Za-z0-9]+)','',tweet)
def removelinks(tweet):
    return re.sub(r'http\S+','',tweet)
def removeemojis(tweet):
    return tweet.encode('ascii', 'ignore').decode('ascii')
def removehashtags(tweet):
    return re.sub(r'(#[A-Za-z0-9]+)','',tweet)
def removePuncs(tweet):
    return re.sub(r'[^\w\s]','', tweet)
tb_df['no_punc_text'] = tb_df['no_punc_text'].apply(removelinks).apply(removeatusernames).apply(removehashtags)
tb_df = tb_df[~tb_df['no_punc_text'].str.startswith(' ')]
tb_df['no_punc_text'] = tb_df['no_punc_text'].str.lower()

from textblob import TextBlob
sentiment_objects = [TextBlob(tweet) for tweet in tb_df['no_punc_text']]
sentiment_values = [[tweet.sentiment.polarity, str(tweet)] for tweet in sentiment_objects]
sentiment_df = pd.DataFrame(sentiment_values, columns=["polarity", "text"])
sentiment_df['sentiment'] = [1 if x >= 0 else 0 for x in sentiment_df['polarity']]

In [ ]:
sentiment 